In [ ]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.ensemble_upperbound import get_ensemble_ub
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml inference_config

log:
    root: '?'
    min_fg_pixels: '?' 
    gether_inference_stats: '?'
    summary_compute_global_metrics: '?' 
    log_interval: 20 
    log_image_stats: True 
    log_pixel_stats: True 
    track_ensemble_member_scores: True 

experiment:
    exp_root: '?'
    seed: 42

data:
    splits: ('cal',)
    preload: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

model:
    calibrator: '?'
    checkpoint: '?' 
    pretrained_exp_root : '?' 
    pretrained_select_metric: "val-dice_score"

In [ ]:
%%yaml ensemble_cfg

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    normalize: False
    member_w_metric: 'None'

In [ ]:
additional_args = {
    "log.gether_inference_stats": [True],
    "log.summary_compute_global_metrics": [True]
}

In [ ]:
# exp_name = '02_25_24_Shapes_BinaryIndSplits' 
# group_dict = {
#     "dataset": "Shapes",
#     "exp_group": exp_name,
#     "base_models_group": "02_25_24_Shapes_BinaryIndependent",
#     "calibrated_models_group": "02_25_24_Shapes_BinaryIndCalibs",
#     "preload": False
# }

exp_name = '02_25_24_Shapes_BinarySharedSplits' 
group_dict = {
    "dataset": "Shapes",
    "exp_group": exp_name,
    "base_models_group": "02_25_24_Shapes_BinaryShared",
    "calibrated_models_group": "02_25_24_Shapes_BinarySharedCalibs",
    "preload": False
}

### Standard Calibrators.

In [ ]:
do_ensemble = True 
inference_func = get_cal_stats
# Get the configs for the different runs.
option_set = get_ese_inference_configs(
    group_dict=group_dict,
    calibrators_list=[
        'Uncalibrated',
        'Vanilla',
        'TempScaling',
        'VectorScaling',
        'DirichletScaling',
        'LTS',
        'NectarScaling',
    ],
    do_ensemble=do_ensemble, 
    additional_args=additional_args
)

### Binning Calibrators

In [ ]:
# do_ensemble = True 
# inference_func = get_cal_stats
# # Get the configs for the different runs.
# option_set = get_ese_inference_configs(
#     group_dict=group_dict,
#     calibrators_list=[
#         'HistogramBinning',
#         'LocalHistogramBinning',
#         'NectarBinning',
#         'SoftNectarBinning'
#     ], 
#     do_ensemble=do_ensemble, 
#     cal_stats_splits=[
#         'val',
#         'cal'
#     ],
#     additional_args=additional_args
# )

### Upper Bounds

In [ ]:
# # # Get the configs for computing the upepr bounds
# inference_func = get_ensemble_ub
# option_set = get_ese_inference_configs(
#     group_dict=group_dict,
#     do_ensemble=True, 
#     ensemble_upper_bound=True,
#     calibrators_list=[
#         'Uncalibrated',
#     ], 
#     ens_cfg_options=[
#         ('upperbound', 'probs')
#     ]
# )

In [ ]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"
##################################################
with open(inf_cfg_root / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)
##################################################
base_cfg = Config(inference_config).update([dataset_inference_cfg, cal_metrics_cfg])
if do_ensemble:
    base_cfg = base_cfg.update(ensemble_cfg)

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [ ]:
base_cfg

In [ ]:
len(cfgs)

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
# from ese.experiment.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     show_examples=True,
#     gpu='3',
# ) 

In [ ]:
from ese.experiment.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    config_list=cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
)